In [1]:
import os
import sys

# Get the current working directory (where the notebook is located)
notebook_dir = os.getcwd()

# Navigate to the parent directory
parent_dir = os.path.dirname(notebook_dir)

# Add the parent directory to the Python path
sys.path.append(parent_dir)

from Reward_Models.Reward_Model_Trajectory import RewardModelTrajectory
from Reward_Models.Reward_Model_Stepwise import RewardModelStepwise
from Agents.Feedback_Agent_Stepwise import FeedbackAgentStepwise
import gymnasium as gym
from Feedback.Feedback_FileHandler import FeedbackFileHandler
import torch as torch

In [2]:
print(torch.cuda.is_available())
def print_q_vals_and_reward(reward_model):
    for s, a in feedback:
        for a_prime in range(4):
            print()
            if a_prime == a:
                print('should be optimal!')
            #print('q-value for state action pair: ', (s, a_prime), 'q: ', feedback_q_table[s][a_prime])
            print('reward for state action pair ', (s, a_prime), 'r: ', reward_model.get_reward(s, a_prime))

True


# Stepwise RM

In [3]:


env_name = 'CliffWalking-v0'
#env_name = "FrozenLake-v1"
learning_rate = 0.1
discount = 0.99
epsilon = 1
num_episodes = 10000
max_steps_per_episode = 100
exploration_decay_rate = 0.01
num_q_updates = 10
feedback_var =  'SRF'
noise = 0
delay = 0.1
batches = 1
start_state = 36 # Cliff_walking
#start_state = 0 # Frozenlake
optimal_val = -13 # Cliff_waling
#optimal_val = 1 # Frozenlake


file_path = os.path.join(parent_dir, rf"Feedback\{env_name}_feedback.txt")

#file_path = rf"C:\Users\PC\Desktop\Uni\Bachelorarbeit\Coding\repo\bt_lascsak\Feedback\{env_name}_feedback.txt"
feedback_filehandler = FeedbackFileHandler(file_path)
feedback = feedback_filehandler.load_dict_from_file()

agent = FeedbackAgentStepwise(env_name, feedback, learning_rate, discount, epsilon,
                              exploration_decay_rate, max_steps_per_episode,
                              num_episodes, num_q_updates, noise, delay, batches, feedback_var)
print(feedback)
env = gym.make(env_name)
num_states = env.observation_space.n
num_actions = env.action_space.n
rm = RewardModelStepwise(num_states, num_actions, batches, learning_rate)
#rm.to('cuda')
states_optimal = [start_state]
states_bad = [start_state, start_state, start_state]
actions_bad = [1, 2, 3]
actions_optimal = [0]
test_trajectory_good = [(start_state, 0)]
test_trajectory_bad = [(start_state, 1), (start_state, 2), (start_state, 3)]
test_trajectory_optimal =[[*feedback]]
for i in range(30):
    rm.update_reward_model(states_bad, actions_bad, [0.0])
    rm.update_reward_model(states_optimal, actions_optimal, [1.0])

#print(rm.get_reward(start_state,0), ' ', rm.get_reward(start_state,1), ' ',rm.get_reward(start_state,2), ' ', rm.get_reward(start_state,3), ' ',)
print(rm.rewards[start_state][:])

{(36, 0): 1, (24, 1): 1, (25, 1): 1, (26, 1): 1, (27, 1): 1, (28, 1): 1, (29, 1): 1, (30, 1): 1, (31, 1): 1, (32, 1): 1, (33, 1): 1, (34, 1): 1, (35, 2): 1}
tensor([ 3.1262, -3.1684, -3.1684, -3.1684], grad_fn=<SliceBackward0>)


# Trajectory RM

In [4]:

file_path = os.path.join(parent_dir, rf"Feedback\{env_name}_feedback.txt")

#file_path = rf"C:\Users\PC\Desktop\Uni\Bachelorarbeit\Coding\repo\bt_lascsak\Feedback\{env_name}_feedback.txt"
feedback_filehandler = FeedbackFileHandler(file_path)
feedback = feedback_filehandler.load_dict_from_file()
print(feedback)
env = gym.make(env_name)
num_states = env.observation_space.n
num_actions = env.action_space.n
rm = RewardModelTrajectory(num_states, num_actions, batches, learning_rate, noise)
#rm.to('cuda')
test_trajectory_good = [[(start_state, 0, 1)]]
feedback_good = [agent._get_feedback_SRF(traj) for traj in test_trajectory_good]
test_trajectory_good = [[(state, action) for traj in test_trajectory_good for state, action, _ in traj]]
print(test_trajectory_good)
print(feedback_good)
test_trajectory_bad = [[(start_state, 1,2), (start_state, 2,2), (start_state, 3,2)]]
feedback_bad = [agent._get_feedback_SRF(traj) for traj in test_trajectory_bad]
test_trajectory_bad = [[(state, action) for traj in test_trajectory_bad for state, action, _ in traj]]

trajectory_decent = list_of_tuples = [[
    (36, 3,1),
    (24, 2,1),
    (25, 1,1),
    (26, 3,1),
    (27, 1,1),
    (28, 0,1),
    (29, 0,1),
    (30, 0,1),
    (31, 1,1),
    (32, 3,1),
    (33, 2,1),
    (34, 1,1),
    (35, 0,1),
    (1,0,1),
    (4,3,1),
    (25,3,1),
    (27,0,1),
    (12,3,1),
    (23,2,1),
    (3,2,1)
]]
feedback_decent =[agent._get_feedback_SRF(traj) for traj in trajectory_decent]
trajectory_decent = [[(state, action) for traj in trajectory_decent for state, action, _ in traj]]


test_trajectory_optimal =[[*feedback]]
test_trajectory_optimal_prime = [[(state, action, 1) for  traj in test_trajectory_optimal for state, action in traj]]
feedback_opt = [agent._get_feedback_SRF(traj) for traj in test_trajectory_optimal_prime]
all_trajs = [test_trajectory_good[0], test_trajectory_bad[0],trajectory_decent[0],test_trajectory_optimal[0]]
all_feedbacks = [feedback_good[0], feedback_bad[0], feedback_decent[0],feedback_opt[0]]


{(36, 0): 1, (24, 1): 1, (25, 1): 1, (26, 1): 1, (27, 1): 1, (28, 1): 1, (29, 1): 1, (30, 1): 1, (31, 1): 1, (32, 1): 1, (33, 1): 1, (34, 1): 1, (35, 2): 1}
[[(36, 0)]]
[1.0]


In [5]:
for i in range(100):
    #rm.update_reward_model(test_trajectory_bad, feedback_bad)
    #rm.update_reward_model(test_trajectory_bad, feedback_bad)
    #rm.update_reward_model(test_trajectory_good, feedback_good)
    #rm.update_reward_model(trajectory_decent, feedback_decent)
    #rm.update_reward_model(test_trajectory_optimal, feedback_opt)
    rm.update_reward_model(all_trajs,all_feedbacks)


print_q_vals_and_reward(rm)


should be optimal!
reward for state action pair  (36, 0) r:  -0.5791217684745789

reward for state action pair  (36, 1) r:  -0.5791217684745789

reward for state action pair  (36, 2) r:  -0.5791217684745789

reward for state action pair  (36, 3) r:  -0.5791217684745789

reward for state action pair  (24, 0) r:  0.0

should be optimal!
reward for state action pair  (24, 1) r:  -0.5791217684745789

reward for state action pair  (24, 2) r:  -0.5791217684745789

reward for state action pair  (24, 3) r:  0.0

reward for state action pair  (25, 0) r:  0.0

should be optimal!
reward for state action pair  (25, 1) r:  -0.5791217684745789

reward for state action pair  (25, 2) r:  0.0

reward for state action pair  (25, 3) r:  -0.5791217684745789

reward for state action pair  (26, 0) r:  0.0

should be optimal!
reward for state action pair  (26, 1) r:  -0.5791217684745789

reward for state action pair  (26, 2) r:  0.0

reward for state action pair  (26, 3) r:  -0.5791217684745789

reward for 

In [6]:
all_trajs.remove(test_trajectory_optimal[0])
all_feedbacks.remove(feedback_opt[0])

for i in range(100):
    #rm.update_reward_model(test_trajectory_bad, feedback_bad)
    #rm.update_reward_model(test_trajectory_bad, feedback_bad)
    #rm.update_reward_model(test_trajectory_good, feedback_good)
    #rm.update_reward_model(trajectory_decent, feedback_decent)
    #rm.update_reward_model(test_trajectory_optimal, feedback_opt)
    rm.update_reward_model(all_trajs,all_feedbacks)



print_q_vals_and_reward(rm)


should be optimal!
reward for state action pair  (36, 0) r:  -0.6020482182502747

reward for state action pair  (36, 1) r:  -1.0077500343322754

reward for state action pair  (36, 2) r:  -1.0077500343322754

reward for state action pair  (36, 3) r:  -1.007750153541565

reward for state action pair  (24, 0) r:  0.0

should be optimal!
reward for state action pair  (24, 1) r:  -0.3326643705368042

reward for state action pair  (24, 2) r:  -1.0077500343322754

reward for state action pair  (24, 3) r:  0.0

reward for state action pair  (25, 0) r:  0.0

should be optimal!
reward for state action pair  (25, 1) r:  -0.6020482182502747

reward for state action pair  (25, 2) r:  0.0

reward for state action pair  (25, 3) r:  -1.0077500343322754

reward for state action pair  (26, 0) r:  0.0

should be optimal!
reward for state action pair  (26, 1) r:  -0.3326643705368042

reward for state action pair  (26, 2) r:  0.0

reward for state action pair  (26, 3) r:  -1.0077500343322754

reward for s

In [7]:
# from a normal training run through
random_traj1 = [(36, 0, 24), (24, 1, 25), (25, 1, 26), (26, 0, 14), (14, 0, 2), (2, 2, 14), (14, 0, 2), (2, 3, 1), (1, 1, 2), (2, 1, 3), (3, 3, 2), (2, 2, 14), (14, 3, 13), (13, 2, 25), (25, 3, 24), (24, 1, 25), (25, 1, 26), (26, 2, 36), (36, 2, 36), (36, 0, 24), (24, 0, 12), (12, 0, 0), (0, 0, 0), (0, 1, 1), (1, 2, 13), (13, 1, 14), (14, 3, 13), (13, 0, 1), (1, 1, 2), (2, 3, 1), (1, 3, 0), (0, 0, 0), (0, 2, 12), (12, 0, 0), (0, 0, 0), (0, 1, 1), (1, 3, 0), (0, 1, 1), (1, 3, 0), (0, 3, 0), (0, 1, 1), (1, 0, 1), (1, 0, 1), (1, 2, 13), (13, 2, 25), (25, 2, 36), (36, 0, 24), (24, 3, 24), (24, 3, 24), (24, 1, 25), (25, 3, 24), (24, 3, 24), (24, 1, 25), (25, 2, 36), (36, 0, 24), (24, 1, 25), (25, 2, 36), (36, 0, 24), (24, 1, 25), (25, 3, 24), (24, 1, 25), (25, 1, 26), (26, 1, 27), (27, 0, 15), (15, 3, 14), (14, 1, 15), (15, 1, 16), (16, 3, 15), (15, 0, 3), (3, 2, 15), (15, 1, 16), (16, 3, 15), (15, 1, 16), (16, 3, 15), (15, 3, 14), (14, 2, 26), (26, 0, 14), (14, 1, 15), (15, 2, 27), (27, 0, 15), (15, 1, 16), (16, 1, 17), (17, 3, 16), (16, 3, 15), (15, 1, 16), (16, 1, 17), (17, 1, 18), (18, 3, 17), (17, 3, 16), (16, 0, 4), (4, 2, 16), (16, 0, 4), (4, 2, 16), (16, 3, 15), (15, 1, 16), (16, 3, 15), (15, 0, 3), (3, 1, 4), (4, 0, 4), (4, 0, 4)]

random_traj2 =[(36, 2, 36), (36, 0, 24), (24, 1, 25), (25, 2, 36), (36, 1, 36), (36, 3, 36), (36, 0, 24), (24, 2, 36), (36, 0, 24), (24, 2, 36), (36, 0, 24), (24, 0, 12), (12, 1, 13), (13, 0, 1), (1, 2, 13), (13, 3, 12), (12, 1, 13), (13, 1, 14), (14, 0, 2), (2, 3, 1), (1, 1, 2), (2, 2, 14), (14, 1, 15), (15, 0, 3), (3, 2, 15), (15, 0, 3), (3, 3, 2), (2, 1, 3), (3, 2, 15), (15, 1, 16), (16, 3, 15), (15, 3, 14), (14, 0, 2), (2, 3, 1), (1, 3, 0), (0, 2, 12), (12, 3, 12), (12, 0, 0), (0, 0, 0), (0, 0, 0), (0, 1, 1), (1, 3, 0), (0, 3, 0), (0, 1, 1), (1, 3, 0), (0, 0, 0), (0, 0, 0), (0, 3, 0), (0, 1, 1), (1, 1, 2), (2, 3, 1), (1, 1, 2), (2, 0, 2), (2, 0, 2), (2, 2, 14), (14, 2, 26), (26, 2, 36), (36, 3, 36), (36, 2, 36), (36, 0, 24), (24, 3, 24), (24, 0, 12), (12, 0, 0), (0, 0, 0), (0, 3, 0), (0, 3, 0), (0, 1, 1), (1, 0, 1), (1, 1, 2), (2, 2, 14), (14, 2, 26), (26, 2, 36), (36, 1, 36), (36, 0, 24), (24, 3, 24), (24, 0, 12), (12, 2, 24), (24, 2, 36), (36, 0, 24), (24, 3, 24), (24, 0, 12), (12, 1, 13), (13, 0, 1), (1, 2, 13), (13, 0, 1), (1, 1, 2), (2, 2, 14), (14, 1, 15), (15, 0, 3), (3, 1, 4), (4, 1, 5), (5, 1, 6), (6, 2, 18), (18, 1, 19), (19, 3, 18), (18, 0, 6), (6, 0, 6), (6, 2, 18), (18, 1, 19), (19, 3, 18)]


rm = RewardModelTrajectory(num_states, num_actions, 3, learning_rate, noise)
random_feedback1 = agent._get_feedback_SRF(random_traj1)
random_feedback2 = agent._get_feedback_SRF(random_traj2)
random_traj1 = [(state,action) for state,action,_ in random_traj1 ]
random_traj2 = [(state,action) for state,action,_ in random_traj2 ]
all_trajs = [random_traj1, test_trajectory_optimal[0], random_traj2]
all_feedbacks = [random_feedback1, feedback_opt[0],random_feedback2]

In [8]:
for i in range(100):
    #rm.update_reward_model(test_trajectory_bad, feedback_bad)
    #rm.update_reward_model(test_trajectory_bad, feedback_bad)
    #rm.update_reward_model(test_trajectory_good, feedback_good)
    #rm.update_reward_model(trajectory_decent, feedback_decent)
    #rm.update_reward_model(test_trajectory_optimal, feedback_opt)
    rm.update_reward_model(all_trajs,all_feedbacks)


print_q_vals_and_reward(rm)


should be optimal!
reward for state action pair  (36, 0) r:  -0.875147819519043

reward for state action pair  (36, 1) r:  -0.8751475811004639

reward for state action pair  (36, 2) r:  -0.875147819519043

reward for state action pair  (36, 3) r:  -0.8751475811004639

reward for state action pair  (24, 0) r:  -0.8751478791236877

should be optimal!
reward for state action pair  (24, 1) r:  -0.875147819519043

reward for state action pair  (24, 2) r:  -0.875147819519043

reward for state action pair  (24, 3) r:  -0.875147819519043

reward for state action pair  (25, 0) r:  0.0

should be optimal!
reward for state action pair  (25, 1) r:  -0.8751478791236877

reward for state action pair  (25, 2) r:  -0.8751478791236877

reward for state action pair  (25, 3) r:  -0.875147819519043

reward for state action pair  (26, 0) r:  -0.8751475811004639

should be optimal!
reward for state action pair  (26, 1) r:  -0.8751475811004639

reward for state action pair  (26, 2) r:  -0.875147819519043

r